# 4DHumans Demo Notebook
This a demo notebook for our paper "4DHumans: Reconstructing and Tracking Humans with Transformers".

<p align="left"><img src="https://github.com/shubham-goel/4D-Humans/raw/main/assets/teaser.png" width="800"></p>

<p align="left"><img src="https://github.com/brjathu/PHALP/raw/master/assets/imgs/teaser.gif" width="800"></p>

Project webpage: https://shubham-goel.github.io/4dhumans/

Github repo: https://github.com/shubham-goel/4D-Humans

Instructions:

1. Enable the GPU Runtime (Runtime > Change Runtime Type > GPU)
2. Clone the repository and install depencencies
3. Run the demo
4. Visualize the video


# Clone the repository and install dependencies

In [ ]:
# Clone the main repo
%%capture
! git clone https://github.com/shubham-goel/4D-Humans.git 4D-Humans
%cd 4D-Humans

In [ ]:
!pip install torch
!pip install -e .[all]
!wget https://github.com/classner/up/raw/master/models/3D/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl
!mkdir data/
!mv basicModel_neutral_lbs_10_207_0_v1.0.0.pkl data/

# Run the demo

In [ ]:
# run hmr2 on a folder of images
!python demo.py \
--img_folder example_data/images \
--out_folder demo_out \
--batch_size=48 --side_view --save_mesh

### Visualize the reconstructions

In [ ]:
from IPython.display import Image, display
import os
# https://colab.research.google.com/drive/1Ex4gE5v1bPR3evfhtG7sDHxQGsWwNwby?usp=sharing
output_images = ["demo_out/" + i for i in os.listdir("demo_out/") if ".png" in i]
for img in output_images:
  display(Image(img))



# Video Demo

### Install PHALP

In [ ]:
!pip install git+https://github.com/brjathu/PHALP.git

### Track and visualize

In [ ]:
!python track.py video.source="example_data/videos/gymnasts.mp4" video.start_frame=10 video.end_frame=60

In [ ]:
# Display the reconstruction video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                      <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

!ffmpeg -y -hide_banner -loglevel error -i outputs/PHALP_gymnasts.mp4 outputs/PHALP_gymnasts_ffmpeg.mp4
show_local_mp4_video('outputs/PHALP_gymnasts_ffmpeg.mp4', width=960, height=540)

# Visualize with Polyscope

In [1]:
import pickle as pkl
import gzip
# joblib is used to load the model outputs
import joblib
import polyscope as ps
import polyscope.imgui as psim
from matplotlib import pyplot as plt
import os, sys
import numpy as np
import cv2
from scipy.interpolate import interp1d

sys.path.insert(0, os.path.abspath("/scratch/ondemand27/evanpan/motion_instruction/smpl"))
sys.path.insert(0, os.path.abspath("/Users/evanpan/Documents/GitHub/Motion_instruction/smpl"))
from smpl_np import SMPLModel
# from smpl.smpl_webuser.serialization import load_model
# from phalp.utils import smpl_utils

In [2]:
# %matplotlib widget
%matplotlib inline
%load_ext autoreload
%autoreload 1

In [3]:

motionData_path = "./4D-Humans_outputs/results/demo_biggerspin1.pkl"
motiondata = joblib.load(open(motionData_path, "rb"))
# load smpl model
smpl = SMPLModel('./smpl/models/model.pkl')

## Load time-varying parameters

In [36]:
# inputs
motionData_path = "./4D-Humans_outputs/results/demo_biggerspin1.pkl"
main_guy = 1

# load motion data
motiondata = joblib.load(open(motionData_path, "rb"))
frames = sorted(list(motiondata.keys()))
per_frame_smpl_data = {"time":[], "beta":[], "pose":[], "position":[]}
for i in range(0, len(frames)):
    frame_i = motiondata[frames[i]]
    # print(frame_i.keys())
    pose_i = frame_i["pose"] # this stores a list of poses for characters in the 
    center_i = frame_i["center"]
    scale_i = frame_i["scale"]
    smpl_i = frame_i["smpl"]
    ids_i = frame_i['tid']
    threeD_joints_i = frame_i['3d_joints']
    # only care about the guy of interest
    if main_guy not in ids_i:
        continue
    pose_i_main = pose_i[ids_i.index(main_guy)]
    threeD_joints_i_main = threeD_joints_i[ids_i.index(main_guy)] # shape is (43, 3)
    smpl_i_main = smpl_i[ids_i.index(main_guy)] # dictionary with keys ['global_orient', 'body_pose', 'betas']
    
    # get the smpl parameters
    global_orient = smpl_i_main["global_orient"] # defined root orentation (rotation matrix )
    body_pose = smpl_i_main["body_pose"] # defines the rotation matrix of the 23 joints (generated using Rodrigues formula)
    beta = smpl_i_main["betas"] # shape is (10,)
    # convert rotation matrices in body pose to axis-angle representation
    body_pose_aa = np.zeros((body_pose.shape[0], 3))
    for jj in range(body_pose.shape[0]):
        body_pose_aa[jj] = cv2.Rodrigues(body_pose[jj])[0].squeeze()
    # rotate global_orient by 180 degrees by x axis
    # global_orient[0] = cv2.Rodrigues(np.array([np.pi, 0, 0]))[0].squeeze() @ global_orient[0]
    global_orient_aa = np.array([cv2.Rodrigues(global_orient[0])[0].squeeze()])
    if len(per_frame_smpl_data["pose"]) > 0:
        # check for discontinuities in the global orientation
        for jj in range(0, len(global_orient_aa[0])):
            if np.abs(global_orient_aa[0][jj] - per_frame_smpl_data["pose"][-1][0, jj]) > np.pi/4:
                global_orient_aa[0][jj] *= -1
    # global_orient_aa[np.where(global_orient_aa > np.pi)] -= 2*np.pi
    # global_orient_aa[np.where(global_orient_aa < -np.pi)] += 2*np.pi
    body_pose_aa = np.concatenate([global_orient_aa, body_pose_aa], axis=0)
    # load data related to smpl model
    per_frame_smpl_data["time"].append(frame_i["time"])
    per_frame_smpl_data["beta"].append(beta)
    per_frame_smpl_data["pose"].append(body_pose_aa)
    per_frame_smpl_data["position"].append(np.zeros([3, ]))
per_frame_smpl_data["beta"] = np.array(per_frame_smpl_data["beta"])
per_frame_smpl_data["pose"] = np.array(per_frame_smpl_data["pose"])
per_frame_smpl_data["position"] = np.array(per_frame_smpl_data["position"])
per_frame_smpl_data["time"] = np.array(per_frame_smpl_data["time"])

per_frame_smpl_data["pose_interp"] = interp1d(per_frame_smpl_data["time"], per_frame_smpl_data["pose"], axis=0, fill_value="extrapolate", bounds_error=False)
per_frame_smpl_data["beta_interp"] = interp1d(per_frame_smpl_data["time"], per_frame_smpl_data["beta"], axis=0, fill_value="extrapolate", bounds_error=False)
per_frame_smpl_data["position_interp"] = interp1d(per_frame_smpl_data["time"], per_frame_smpl_data["position"], axis=0, fill_value="extrapolate", bounds_error=False)
# 

# print(per_frame_smpl_data["beta"].shape, per_frame_smpl_data["pose"].shape, per_frame_smpl_data["position"].shape)

## Visualize detected mesh

In [42]:
def update_mesh():
    global mesh_list, params_list, simulator_t
    for i in range(0, len(mesh_list)):
        beta = params_list[i]["beta_interp"](simulator_t)
        pose = params_list[i]["pose_interp"](simulator_t)
        position = params_list[i]["position_interp"](simulator_t)
        smpl.set_params(beta=beta, pose=pose, trans=position)
        mesh_list[i].update_vertex_positions(smpl.verts)
# SM1.update_vertex_positions(V1)
def guii():
    global simulator_t, simulator_to_end_t
    # psim.TextUnformatted("Some sample text")
    time_changed, simulator_t = psim.SliderFloat("time", simulator_t, v_min=0, v_max=simulator_to_end_t)
    if time_changed:
        update_mesh()
params_list = [per_frame_smpl_data]
mesh_list = []
ps.set_verbosity(0)
ps.set_SSAA_factor(4)
ps.set_program_name("Interactive Viewer")
ps.set_ground_plane_mode("none")
ps.set_view_projection_mode("orthographic")
ps.set_autocenter_structures(False)
ps.set_autoscale_structures(False)
ps.set_front_dir("z_front")
ps.set_up_dir("neg_y_up")
ps.set_background_color([0, 0, 0])
ps.init()
simulator_t = 0
simulator_to_end_t = params_list[0]["time"][-1]
# iterate through each clip to create the mesh + load the parameters
for i in range(0, len(params_list)):
    params = params_list[i]
    smpl.set_params(beta=params["beta"][0], pose=params["pose"][0], trans=params["position"][0])
    SM_temp = ps.register_surface_mesh("original", smpl.verts, smpl.faces, color=[0.9,0.9,0.9], smooth_shade=True, edge_width=0.25, )
    mesh_list.append(SM_temp)

ps.set_user_callback(guii)
ps.show()